# Un détecteur de distanciation sociale COVID-19

#### Réalisé par : Mariam Loukili

* Importer les modèles suivants : 

In [64]:
from scipy.spatial import distance as dist
import numpy as np
import imutils
import cv2

* Fixer les seuils.

In [65]:
min_conf = 0.5
min_distance = 60
nms_thresh = 0.4

* Définir la méthode people_detection() pour détecter seulement les personnes.

In [67]:
def people_detection(img, net, classes) :
    #Extraire la hauteu et la largeur et initialiser la liste results
    height, width, _ = img.shape
    results = []
    
    blob = cv2.dnn.blobFromImage(img, 1/255, (608,608), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layersOutputs = net.forward(output_layers_names)
    
   # Initialiser les listes
    boxes = []
    confidences = []
    centroids = []
    class_ids = []
    
    # Boucler sur chacune de layer Outputs 
    for output in layersOutputs:
        # Boucler sur chacune des détections
        for detection in output:
            # Extraire la classe ID et la confiance (c'est-à-dire la probabilité) de la détection de l'objet en cours
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            # Vérifier que le minimum de confiance est respecté
            if confidence > min_conf:
                
                # En tenant compte que YOLO renvoie en fait les coordonnées du centre (x, y) du bounding box,
                #suivies de la largeur et de la hauteur du cadre d'objet.
                center_x =  int(detection[0]*width)
                center_y =  int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                
                # Utiliser les coordonnées du centre (x, y) pour obtenir le coin supérieur et le coin gauche du cadre d'objet.
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                
                # Mettre à jour les listes de coordonnées de bounding box, classe IDs ,de centroïdes et de confidences
                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)
                centroids.append((center_x,center_y))
                
    # Appliquer une suppression non-maxima pour supprimer les bounding box faibles.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_conf, nms_thresh)
    
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))
    
    # Boucler sur les index que nous gardons
    for i in indexes.flatten():
        x,y,w,h = boxes[i]
        label= str(classes[class_ids[i]])
        
        # En s'assurant que l'objet détecté est une personne 
        if label == 'person': 
            confidence =str(round(confidences[i]*100, 2))
            
            # Mettre à jour la liste "result" pour qu'elle comprenne la probabilité de prédiction de la personne, 
            #les coordonnées de bounding boxet les centroïdes. 
            r = (confidence, (x, y, x+w, y+h), centroids[i])
            results.append(r)
            
    # Retourner la liste des résultats
    return results

*  Calculer la distance entre chaque couple de personnes. Puis vérifier si la distance est inférieure au minimum distance.

In [73]:
#Charger notre détecteur d'objets YOLOv3 formé sur le dataset COCO (80 classes)
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
# Extraire les noms des objets du fichier coco et les ajouter à la liste "classes".
classes = []
with open('coco.names', 'r') as f:
    classes = f.read().splitlines()

# Lire la video
print("[INFO] accessing video stream...")
cap = cv2.VideoCapture('video.mp4')

# En boucle sur les images de la vidéo !
while True:
    # lire l'image suivante du fichier
    result, img = cap.read()
    if result==False:  # tester s'il y a une image ou non dans la video
        print("Fin de la video")
        break

    #  redimensionner l'image et appeler la méthode people_detection() 
    img = imutils.resize(img, width=900)
    results = people_detection(img, net, classes)

    # Initialiser l'ensemble des gens qui sont proches les uns des autres, sans respecter la distance sociale minimum.  
    proche = set()

    # S'assurer qu'il y a au moins deux personnes détectées (nécessaire pour calculer la distance)
    if len(results) >= 2:
        # Extraire tous les centroïdes de la list "results" et calculer les distances euclidiennes entre toutes
        #les paires de centroïdes.
        centroids = np.array([r[2] for r in results])
        D = dist.cdist(centroids, centroids, metric="euclidean")

        for i in range(0, D.shape[0]):
            for j in range(i + 1, D.shape[1]):
                # Vérifier si la distance entre deux paires de centroïdes est inférieure au nombre de pixels configuré
                if D[i, j] < min_distance:
                    # Mettre à jour notre set "proche" avec les index des paires de centroïdes.
                    proche.add(i)
                    proche.add(j)

    # Boucler sur "result"
    for (i, (prob, bbox, centroid)) in enumerate(results):
        # Extraire les bounding box et les coordonnées du centroïde, puis initialiser la couleur de l'annotation
        (startX, startY, endX, endY) = bbox
        (cX, cY) = centroid
        color = (0, 255, 0)

        # Si la paire d'indices existe dans l'ensemble des personnes proches, alors modifiez la couleur.
        if i in proche:
            color = (0, 0, 255)

        # Dessiner bounding box autour de la personne.
        cv2.rectangle(img, (startX, startY), (endX, endY), color, 2)
        # Dessiner le centroïde de la personne.
        cv2.circle(img, (cX, cY), 2, color, 1)

    # Afficher le nombre total des gens qui violent la distance sociale
    text = "Distanciation sociale insuffisante {}".format(len(proche))
    cv2.putText(img, text, (5, img.shape[0] - 25),
        cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 2)

   # Afficher les résultats sur l'écran
    cv2.imshow("Frame", img)
    key = cv2.waitKey(1) & 0xFF

    # si la touche "Entrée" a été appuyée, sortir de la boucle.
    if key == 13:
        print("Fin avec Entrer")
        break

cap.release()
cv2.destroyAllWindows()

[INFO] accessing video stream...
Fin avec Entrer
